# Amazon ESCI dataset EDA

The goal of this notebook is to identify how this dataset was created, any interesting features, benchmarks, and metrics used to evaluate it. Also, some simple EDA was performed to see distribution of features

In [163]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from ranx import Qrels, Run, evaluate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [164]:
df_examples = pd.read_parquet('../data/shopping_queries_dataset_examples.parquet')
df_products = pd.read_parquet('../data/shopping_queries_dataset_products.parquet')
df_sources = pd.read_csv("../data/shopping_queries_dataset_sources.csv")

In [165]:
# https://github.com/amazon-science/esci-data: suggested filter for task 1: Query-Product Ranking 
# Query-Product Ranking: Given a user specified query and a list of matched products, the goal of this 
# task is to rank the products so that the relevant products are ranked above the non-relevant ones.
df_examples_products = pd.merge(
    df_examples,
    df_products,
    how='left',
    left_on=['product_locale','product_id'],
    right_on=['product_locale', 'product_id']
)

df_task_1 = df_examples_products[df_examples_products["small_version"] == 1]
df_task_1_train = df_task_1[df_task_1["split"] == "train"]
df_task_1_test = df_task_1[df_task_1["split"] == "test"]

# simple EDA

In [166]:
# describe
df_task_1.describe()

,example_id,query_id,small_version,large_version
count,1.118011e+06,1.118011e+06,1118011.0,1118011.0
mean,1.376919e+06,6.963481e+04,1.0,1.0
std,8.195697e+05,4.190752e+04,0.0,0.0
min,1.600000e+01,1.000000e+00,1.0,1.0
25%,6.453855e+05,3.202900e+04,1.0,1.0
50%,1.405883e+06,7.142900e+04,1.0,1.0
75%,2.159588e+06,1.106680e+05,1.0,1.0
max,2.621255e+06,1.306490e+05,1.0,1.0


In [167]:
# split of queries per product location
df_task_1.product_locale.value_counts()

product_locale
us    601354
jp    297883
es    218774
Name: count, dtype: int64

In [168]:
# train-test split
print("Train rows count percentage", len(df_task_1_train)/len(df_task_1))
print("Test rows count percentage", len(df_task_1_test)/len(df_task_1))
print("Train queries count percentage", df_task_1_train['query_id'].nunique()/df_task_1['query_id'].nunique())
print("Test queries count percentage", df_task_1_test['query_id'].nunique()/df_task_1['query_id'].nunique())

Train rows count percentage 0.6991326561187681
Test rows count percentage 0.30086734388123193
Train queries count percentage 0.6998757763975155
Test queries count percentage 0.3001242236024845


### Align values to dataset description
Check the counts match what was shown on: https://github.com/amazon-science/esci-data
![reduced-dataset-count-product-locale](imgs/dataset_total.png)

### Dataset understanding

Each query_id is unique to a user search. \
Each judgement is a product that got manually evaluated per query_id. For e.g. 35 products were shown to the user for a given query. \
The depth is the count of the number of products that were evaluated per query.

In [169]:
# average queries per judgement
print("Average judgements per query", df_task_1.groupby('query_id')['example_id'].nunique().mean())
print("Max judgements per query", df_task_1.groupby('query_id')['example_id'].nunique().max())
print("Min judgements per query", df_task_1.groupby('query_id')['example_id'].nunique().min())

Average judgements per query 23.14722567287785
Max judgements per query 188
Min judgements per query 8


### Data cleaning
Identify duplicate query-title pairs, potential bad labels, bad queries, really short queries and products in the training dataset.

In [173]:
duplicates = df_task_1_train[df_task_1_train.duplicated(subset=["query", "product_title"])]
duplicates

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color,query_length
2153,2153,$10 magnetic eyelashes without eyeliner,70,B08MD4LTJ2,us,S,1,1,train,EARLLER 10 Pairs Magnetic Eyelashes with Eyeli...,None,"Magnetic Eyeliner : Rejecting glue, EARLLER ma...",Earller,None,5
5693,5693,(éxito y productividad sin límites,189,1072076810,es,S,1,1,train,Gestión del tiempo: Productividad personal sin...,None,None,None,None,5
6040,6040,*i'm gaming do not disturb*shirt,200,B07XRYSHBG,us,E,1,1,train,Do Not Disturb I'm Gaming T-Shirt Video Gamer ...,None,Perfect Birthday Gift Idea for Men / Women / K...,Do Not Disturb I'm Gaming Shirts,Black,5
6572,6572,+crunchy peanuts not butter,217,B0046HNS8Y,us,I,1,1,train,Santa Cruz Organic Crunchy Dark Roasted Peanut...,None,Crunchy peanut butter made from certified USDA...,Santa Cruz ORGANIC,None,4
7453,7453,", máquina de depilación sin dolor",246,B089ND86RN,es,E,1,1,train,Depiladora de Luz Pulsada para Mujeres Maquina...,None,【Depilación Permanente Efectiva】El depiladora ...,eme,None,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2620848,2620848,ｚａｒｄ 負けないで,130631,B009XH9QC4,jp,S,1,1,train,負けないで,None,None,None,None,2
2620860,2620860,ｚａｒｄ 負けないで,130631,B089B2RKKG,jp,C,1,1,train,負けないで(オリジナルアーティスト:ZARD)[ガイドメロディ無しカラオケ],None,None,None,None,2
2620875,2620875,ｚａｒｄ 負けないで,130631,B00005F59G,jp,E,1,1,train,負けないで,None,None,None,None,2
2620934,2620934,ｺｽﾓｽ,130635,B0784P282F,jp,E,1,1,train,HGBF ガンダムビルドファイターズ バトローグ ビルドストライクギャラクシーコスモス 1/...,None,(C)創通・サンライズ\n<b>対象年齢 :</b>8才以上,バンダイ(BANDAI),None,1


In [174]:
# Check if any values in query, title, description are nan
print(df_task_1.isna().any())


example_id              False
query                   False
query_id                False
product_id              False
product_locale          False
esci_label              False
small_version           False
large_version           False
split                   False
product_title           False
product_description      True
product_bullet_point     True
product_brand            True
product_color            True
dtype: bool


In [183]:
nan_desc = df_task_1[df_task_1['product_description'].isna()]
product_nan = len(nan_desc['product_description']) / len(df_task_1)
perc_product_nan = product_nan*100
print(f"{perc_product_nan:.2f}% of product descriptions are None")

51.75% of product descriptions are None


In [171]:
# plot distribution of query length
df_task_1_train["query_length"] = df_task_1_train["query"].str.split().str.len()

# count words
length_counts = df_task_1_train["query_length"].value_counts().sort_index()

fig = go.Figure(
    data=[
        go.Bar(
            x=length_counts.index,
            y=length_counts.values,
        )
    ]
)

fig.update_layout(
    title="Distribution of query words",
    xaxis_title="Number of words in query",
    yaxis_title="Frequency",
)

fig.show()

C:\Users\ellen\AppData\Local\Temp\ipykernel_26464\2806754784.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### ESCI understanding
- E: Exact
- S: Substitute
- C: Complement
- I: Irrelevant

These provide rough ranks for the judgements per query_id.

In [ ]:
# calculate the average esci ratio for each query_id
esci_counts = df_task_1.groupby(['query_id', 'esci_label']).size().unstack(fill_value=0)
esci_ratios = esci_counts.div(esci_counts.sum(axis=1), axis=0)
avg_esci_ratio = esci_ratios.mean(axis=0)

avg_esci_ratio

esci_label
C    0.053341
E    0.437810
I    0.163013
S    0.345836
dtype: float64

### Easy metric evaluation 

Using a simple cosine_similarity, calculate the ndcg between query-title using the ESCI weightings


In [ ]:
one_query.head()

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
32,32,!qscreen fence without holes,2,B07D7TBSGH,us,I,1,1,test,FOTMISHU 6Pcs Greenhouse Hoops Rust-Free Grow ...,<p><b>Description:</b><br><b>Material:</b>plas...,►Material -- High-quality plastic coated steel...,FOTMISHU,green
33,33,!qscreen fence without holes,2,B07DHT2WZK,us,I,1,1,test,Zippity Outdoor Products ZP19028 Unassembled M...,None,Designed for the Madison Vinyl Fence (item ZP1...,Zippity Outdoor Products,White
34,34,!qscreen fence without holes,2,B07DHX8YH2,us,E,1,1,test,Zippity Outdoor Products ZP19026 Lightweight P...,None,Designed as a portable fence for temporary eve...,Zippity Outdoor Products,White
35,35,!qscreen fence without holes,2,B07DS1YCRZ,us,S,1,1,test,ColourTree 4' x 50' Green Fence Privacy Screen...,None,"★【Virgin, UV Stabilized Compounds & Thick Mate...",ColourTree,Green 2nd Generation
36,36,!qscreen fence without holes,2,B07DS3J3MB,us,S,1,1,test,ColourTree 6' x 50' Black Fence Privacy Screen...,None,"★【Virgin, UV Stabilized Compounds & Thick Mate...",ColourTree,Black 2nd Generation


In [151]:
one_query = df_task_1[df_task_1['query_id'] == 2]

# calculate cosine similarity between pairs of query-title
vectorizer = TfidfVectorizer()

def calculate_cosine_similarity(query, title):
    vectors = vectorizer.fit_transform([query, title])
    cosine_sim = cosine_similarity(vectors[0], vectors[1])
    return cosine_sim.item()

one_query['cosine_similarity'] = one_query.apply(lambda row: calculate_cosine_similarity(
    row['query'], 
    row['product_title']), 
    axis=1)

# apply esci mapping to esci label
esci_weighting = {
    'E': 3,
    'S': 2,
    'C': 1,
    'I': 0
}

one_query['relevance'] = one_query['esci_label'].map(esci_weighting)

# use ranx system to calculate ndcg
qrels_dict = {}
run_dict = {}

for query_id, group in one_query.groupby("query_id"):
    query_id_str = str(query_id)
    qrels_dict[query_id_str] = {str(example): int(relevance) for example, relevance in zip(group["example_id"], group["relevance"])}
    run_dict[query_id_str] = {str(example): float(score) for example, score in zip(group["example_id"], group["cosine_similarity"])}

qrels = Qrels(qrels_dict)
run = Run(run_dict)

results = evaluate(qrels, run, metrics="ndcg@10")
results

C:\Users\ellen\AppData\Local\Temp\ipykernel_26464\1639211472.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ellen\AppData\Local\Temp\ipykernel_26464\1639211472.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



np.float64(0.3881064391080875)

### Model evaluation
Amazon trained and finetuned a BERT model to evaluate on the amazon-esci dataset. They use this dataset for three use cases - query product ranking, multiclass product classification and product substitute identification. For their first use case, they fine tuned a MS MARCO Cross-Encoder for the us locale. For their es and jp locales, they finetuned a multilingual MPNet. 

![dataset-benchmark](imgs/amazon_finetune_results.png)